<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250413%5D%20(FedProx)%20FL%20MLP-LSTM%20on%20WSN-BFSF%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FedProx-FL // with clean MLP-LSTM model 4 classes on WSN-BFSF**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Flatten
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.utils import to_categorical

In [ ]:
# Load dataset
dataset = pd.read_csv("dataset.csv") #WSN-BFSF 4 classes dataset

# Preprocessing: Handle missing values (if any)
dataset = dataset.dropna() # Remove missing values

In [ ]:
dataset

,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num,Class
0,1,0.100000,79,79,600.000000,5,0,79.255,1.255,48,30,1,1,100,0,79,4,0
1,2,0.100963,78,78,599.979723,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
2,2,0.100963,76,76,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
3,2,0.100964,75,75,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
4,2,0.100964,118,118,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312101,4,656.526536,140,140,465.332078,5,800,140.255,1.255,48,30,1,31,100,16,140,68,1
312102,4,657.209643,140,140,465.191806,5,800,140.255,1.255,48,30,1,33,100,16,140,72,1
312103,4,657.891729,140,140,465.051737,5,800,140.255,1.255,48,30,1,35,100,16,140,76,1
312104,4,658.574656,140,140,464.911501,5,800,140.255,1.255,48,30,1,37,100,16,140,80,1


**Data Preprocessing**

In [ ]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

In [ ]:
print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [262851  11766   7645  29844]


In [ ]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
!pip install -U imbalanced-learn

**Data Balancing utilized SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE
# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [262851 262851 262851 262851]


**Split data into 80% Training and 20% Testing**

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

**The hybrid MLP + LSTM model**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, TimeDistributed, InputLayer
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

# Convert target to categorical (one-hot encoding)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Reshape the input data for LSTM compatibility
# Assuming the input features need to be reshaped into a 3D array: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the hybrid MLP + LSTM model with regularization
def build_hybrid_model(input_shape, num_classes):
    model = Sequential()

    # Input Layer
    model.add(InputLayer(input_shape=input_shape))

    # LSTM Layer
    model.add(LSTM(64, activation='tanh', return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization

    # MLP Layers
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile the model
input_shape = (X_train.shape[1], X_train.shape[2])  # Adjusted for LSTM input format
num_classes = y_train.shape[1]
model = build_hybrid_model(input_shape, num_classes)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


26286/26286 ━━━━━━━━━━━━━━━━━━━━ 171s 6ms/step - accuracy: 0.7812 - loss: 0.6250 - val_accuracy: 0.8830 - val_loss: 0.3312
Epoch 2/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 178s 7ms/step - accuracy: 0.9064 - loss: 0.2956 - val_accuracy: 0.9216 - val_loss: 0.2355
Epoch 3/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 175s 7ms/step - accuracy: 0.9233 - loss: 0.2478 - val_accuracy: 0.9255 - val_loss: 0.2235
Epoch 4/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 182s 7ms/step - accuracy: 0.9344 - loss: 0.2212 - val_accuracy: 0.9447 - val_loss: 0.1787
Epoch 5/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 178s 7ms/step - accuracy: 0.9425 - loss: 0.2010 - val_accuracy: 0.9343 - val_loss: 0.2130
Epoch 6/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 178s 7ms/step - accuracy: 0.9398 - loss: 0.2083 - val_accuracy: 0.9624 - val_loss: 0.1467
Epoch 7/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 184s 7ms/step - accuracy: 0.9537 - loss: 0.1726 - val_accuracy: 0.9561 - val_loss: 0.1540
Epoch 8/10
26286/26286 ━━━━━━━━━━━━━━━━━━━━ 175s 7ms/step - accuracy: 0.9

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
# Metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("Accuracy: {:.10f}".format(accuracy))
print("Precision: {:.10f}".format(precision))
print("Recall: {:.10f}".format(recall))
print("F1-Score: {:.10f}".format(f1))

Accuracy: 0.9750238966
Precision: 0.9751185269
Recall: 0.9750238966
F1-Score: 0.9749834268


In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("Accuracy: {:.10f}".format(accuracy))
print("Precision: {:.10f}".format(precision))
print("Recall: {:.10f}".format(recall))
print("F1-Score: {:.10f}".format(f1))

6572/6572 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step
Accuracy: 0.9750238966
Precision: 0.9751185269
Recall: 0.9750238966
F1-Score: 0.9749834268


**FedProx Training**

In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Function to measure model performance
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)

    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = 0
    for i in range(len(local_weights)):
        communication_cost += sum(w.nbytes for w in local_weights[i])
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")  # in MB

# FedProx Local Training Function
def train_fedprox(local_model, X, y, global_weights, mu=0.01, epochs=1, batch_size=16):
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    train_dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

    for epoch in range(epochs):
        for step, (batch_x, batch_y) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                predictions = local_model(batch_x, training=True)
                ce_loss = loss_fn(batch_y, predictions)
                prox_term = tf.add_n([tf.reduce_sum(tf.square(w - gw))
                                      for w, gw in zip(local_model.trainable_weights, global_weights)])
                total_loss = ce_loss + (mu / 2.0) * prox_term
            grads = tape.gradient(total_loss, local_model.trainable_weights)
            optimizer.apply_gradients(zip(grads, local_model.trainable_weights))
    return local_model.get_weights()

# Federated Learning Simulation (FedProx)
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)

    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()

    latency_per_round = []

    for round in range(25):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            updated_weights = train_fedprox(local_model, client_data[i], client_labels[i], global_weights, mu=0.01)
            local_weights.append(updated_weights)

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Note: Make sure build_hybrid_model, input_shape, and num_classes are defined in your environment before running.
# Call the federated learning simulation function
federated_learning_simulation(X_train, y_train, X_test, y_test)


Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.7021319270
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step

Model Performance Metrics:
Accuracy: 0.7021319092
Precision: 0.7089918056
Recall: 0.7021319092
F1-Score: 0.6834072408

Confusion Matrix:
[[32380  5767 12892  1531]
 [12632 20031 13619  6288]
 [ 2881   748 42663  6278]
 [    0     0     0 52571]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.8205115795
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step

Model Performance Metrics:
Accuracy: 0.8205116011
Precision: 0.8179056748
Recall: 0.8205116011
F1-Score: 0.8157209448

Confusion Matrix:
[[37973  7915  6541   141]
 [ 8910 35432  2748  5480]
 [ 1109   704 47237  3520]
 [    0     0   675 51896]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 3: Global Model Accuracy = 0.8409556746
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step

Model Performance Metrics:
Accuracy: 0.8409556736
Precision: 0.8376325712
Recall: 0.8409556736
F1-Score: 0.8368240693

Confusion Matrix:
[[38311  8272  5906    81]
 [ 8364 37398  1395  5413]
 [  756   675 49535  1604]
 [    0     0   978 51593]]


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 13: Global Model Accuracy = 0.8917828798
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

Model Performance Metrics:
Accuracy: 0.8917829000
Precision: 0.9005809684
Recall: 0.8917829000
F1-Score: 0.8892389095

Confusion Matrix:
[[37546 11397  3627     0]
 [  367 46717   102  5384]
 [   55   107 51268  1140]
 [    0     0   577 51994]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 14: Global Model Accuracy = 0.9016268849
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step

Model Performance Metrics:
Accuracy: 0.9016268707
Precision: 0.9089769024
Recall: 0.9016268707
F1-Score: 0.8998279588

Confusion Matrix:
[[39348  9994  3228     0]
 [  175 46943    78  5374]
 [   86   101 51243  1140]
 [    0     0   510 52061]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
!pip install cleverhans

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.9 MB/s eta 0:00:00


**Injection adversarial FGSM Attacks**

In [ ]:
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method # Import the actual function
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent # Import the actual function

# Adversarial config
USE_ADVERSARIAL = True
ATTACK_TYPE = 'fgsm'  # or 'pgd'
ADV_CLIENTS = [0, 2]  # clients to inject adversarial samples
EPSILON = 0.1

# Function to generate FGSM or PGD adversarial examples
def generate_adversarial(model, x, y, attack_type='fgsm', eps=0.1):
    if attack_type == 'fgsm':
        return fast_gradient_method(model_fn=model, x=x, eps=eps, norm=np.inf) # Call with correct arguments
    elif attack_type == 'pgd':
        return projected_gradient_descent(model_fn=model, x=x, eps=eps, eps_iter=0.01, nb_iter=40, norm=np.inf) # Call with correct arguments
    return x

# Function to evaluate the global model
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)
    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = sum(sum(w.nbytes for w in lw) for lw in local_weights)
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")

# Federated Learning Simulation with optional adversarial injection
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)
    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()
    communication_overhead = []
    latency_per_round = []

    for round in range(30):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Inject adversarial examples if enabled
            if USE_ADVERSARIAL and i in ADV_CLIENTS:
                print(f"Injecting {ATTACK_TYPE.upper()} adversarial samples into Client {i}")
                client_data[i] = generate_adversarial(local_model, client_data[i], client_labels[i], attack_type=ATTACK_TYPE, eps=EPSILON)

            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0)
            local_weights.append(local_model.get_weights())

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Call the simulation
federated_learning_simulation(X_train, y_train, X_test, y_test)

Injecting FGSM adversarial samples into Client 0
Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.5813411474
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.5813411578
Precision: 0.5888156288
Recall: 0.5813411578
F1-Score: 0.5653783842

Confusion Matrix:
[[23962 19986  5717  2905]
 [14051 28350  3472  6697]
 [ 7624 20786 17630  6530]
 [    0   268     0 52303]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.8288718462
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step

Model Performance Metrics:
Accuracy: 0.8288718429
Precision: 0.8269974513
Recall: 0.8288718429
F1-Score: 0.8251439134

Confusion Matrix:
[[38415  8640  5487    28]
 [ 8176 37424  1650  5320]
 [ 1105   413 46377  4675]
 [    0     0   491 52080]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2


...

Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 28: Global Model Accuracy = 0.8299703598
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step

Model Performance Metrics:
Accuracy: 0.8299703730
Precision: 0.8311531330
Recall: 0.8299703730
F1-Score: 0.8282944605

Confusion Matrix:
[[41251  6902  4406    11]
 [12402 36627    18  3523]
 [ 3246   266 45850  3208]
 [    0   245  1527 50799]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 29: Global Model Accuracy = 0.8504667282
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.8504667564
Precision: 0.8534695773
Recall: 0.8504667564
F1-Score: 0.8490733298

Confusion Matrix:
[[43508  5906  3130    26]
 [12408 36814     4  3344]
 [ 3177    56 47515  1822]
 [    0   202  1369 51000]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 30: Global Model Accuracy = 0.8141629696
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.8141629534
Precision: 0.8265583717
Recall: 0.8141629534
F1-Score: 0.8136464523

Confusion Matrix:
[[43509  5892  3151    18]
 [13215 35218    41  4096]
 [ 8592    56 41081  2841]
 [    0   142  1034 51395]]

Latency Analysis:
Average Latency per Round: 262.27 seconds
Total Latency for 50 Rounds: 7868.20 seconds


**Injection adversarial PGD Attacks**

In [ ]:
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method # Import the actual function
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent # Import the actual function

# Adversarial config
USE_ADVERSARIAL = True
ATTACK_TYPE = 'pgd'  # or 'pgd'
ADV_CLIENTS = [0, 2]  # clients to inject adversarial samples
EPSILON = 0.1

# Function to generate FGSM or PGD adversarial examples
def generate_adversarial(model, x, y, attack_type='fgsm', eps=0.1):
    if attack_type == 'fgsm':
        return fast_gradient_method(model_fn=model, x=x, eps=eps, norm=np.inf) # Call with correct arguments
    elif attack_type == 'pgd':
        return projected_gradient_descent(model_fn=model, x=x, eps=eps, eps_iter=0.01, nb_iter=40, norm=np.inf) # Call with correct arguments
    return x

# Function to evaluate the global model
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)
    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = sum(sum(w.nbytes for w in lw) for lw in local_weights)
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")

# Federated Learning Simulation with optional adversarial injection
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)
    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()
    communication_overhead = []
    latency_per_round = []

    for round in range(30):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Inject adversarial examples if enabled
            if USE_ADVERSARIAL and i in ADV_CLIENTS:
                print(f"Injecting {ATTACK_TYPE.upper()} adversarial samples into Client {i}")
                client_data[i] = generate_adversarial(local_model, client_data[i], client_labels[i], attack_type=ATTACK_TYPE, eps=EPSILON)

            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0)
            local_weights.append(local_model.get_weights())

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Call the simulation
federated_learning_simulation(X_train, y_train, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 0
Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.5783832073
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step

Model Performance Metrics:
Accuracy: 0.5783832110
Precision: 0.5816343536
Recall: 0.5783832110
F1-Score: 0.5657749722

Confusion Matrix:
[[27050 18868  6309   343]
 [15079 26321  5535  5635]
 [28859  3254 16078  4379]
 [    0     3   394 52174]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.7876080275
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.7876080102
Precision: 0.7864865358
Recall: 0.7876080102
F1-Score: 0.7839969959

Confusion Matrix:
[[37849 10196  4525     0]
 [12428 32861  1909  5372]
 [ 6099   838 42540  3093]
 [    0    73   129 52369]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 28: Global Model Accuracy = 0.8159034848
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

Model Performance Metrics:
Accuracy: 0.8159034815
Precision: 0.8231404617
Recall: 0.8159034815
F1-Score: 0.8140360637

Confusion Matrix:
[[42614  6511  3347    98]
 [13199 33879   108  5384]
 [ 6528   159 43827  2056]
 [    0     0  1322 51249]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 29: Global Model Accuracy = 0.8075004220
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

Model Performance Metrics:
Accuracy: 0.8075004399
Precision: 0.8145951377
Recall: 0.8075004399
F1-Score: 0.8051759324

Confusion Matrix:
[[42493  6536  3376   165]
 [13227 34031    81  5231]
 [ 5353   237 42229  4751]
 [    0    88  1434 51049]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 30: Global Model Accuracy = 0.8040146232
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step

Model Performance Metrics:
Accuracy: 0.8040146280
Precision: 0.8138339841
Recall: 0.8040146280
F1-Score: 0.8022400998

Confusion Matrix:
[[42666  6235  3560   109]
 [13511 34004    65  4990]
 [ 7194   189 41209  3978]
 [    0    73  1308 51190]]

Latency Analysis:
Average Latency per Round: 686.68 seconds
Total Latency for 50 Rounds: 20600.35 seconds
